In [1]:
import tensorflow as tf
import quickdraw as qd
import os
import matplotlib.pyplot as plt
import pickle

In [2]:
input = qd.QuickDrawData()
input_data = []

In [3]:
def get_data(name, max_drawings, recognized):
    drawings = qd.QuickDrawDataGroup(name, max_drawings=max_drawings, recognized=recognized)
    for drawing in drawings.drawings:
        input_data.append([name, drawing.image_data])

In [4]:
def get_images(data, count, label, number):
    x = []
    y = []
    for strokes in data:
        xs = strokes[0]
        ys = strokes[1]
        x.append(xs)
        y.append(ys)
    plt.figure(figsize = (0.32, 0.32), dpi = 100)
    for i in range(len(x)):
        plt.plot(x[i], y[i], color = "black")
    plt.gca().invert_yaxis()
    plt.grid(False)
    plt.axis("off")
    if not os.path.isdir(f"google_{number}_image_images/{label}"):
        os.makedirs(f"google_{number}_image_images/{label}")
    plt.savefig(f"google_{number}_image_images/{label}/{count}.png")
    plt.clf()
    plt.close()

In [5]:
def get_custom_images(data, count, label, number):
    x = []
    y = []
    for strokes in data:
        xs = strokes[0]
        ys = strokes[1]
        x.append(xs)
        y.append(ys)
    plt.figure(figsize = (0.32, 0.32), dpi = 100)
    for i in range(len(x)):
        plt.plot(x[i], y[i], color = "black")
    plt.gca().invert_yaxis()
    plt.grid(False)
    plt.axis("off")
    if not os.path.isdir(f"custom_{number}_image_images/{label}"):
        os.makedirs(f"custom_{number}_image_images/{label}")
    plt.savefig(f"custom_{number}_image_images/{label}/{count}.png")
    plt.clf()
    plt.close()

In [6]:
number = 100
i = 1
for label in qd.QuickDrawData().drawing_names:
    print(f"{i} / 345")
    get_data(label, max_drawings=number, recognized=True)
    i += 1

with open (f"google_{number}_image_stroke_data", "wb") as file:
    pickle.dump(input_data, file)

1 / 345
loading aircraft carrier drawings
load complete
2 / 345
loading airplane drawings
load complete
3 / 345
loading alarm clock drawings
load complete
4 / 345
loading ambulance drawings
load complete
5 / 345
loading angel drawings
load complete
6 / 345
loading animal migration drawings
load complete
7 / 345
loading ant drawings
load complete
8 / 345
loading anvil drawings
load complete
9 / 345
loading apple drawings
load complete
10 / 345
loading arm drawings
load complete
11 / 345
loading asparagus drawings
load complete
12 / 345
loading axe drawings
load complete
13 / 345
loading backpack drawings
load complete
14 / 345
loading banana drawings
load complete
15 / 345
loading bandage drawings
load complete
16 / 345
loading barn drawings
load complete
17 / 345
loading baseball bat drawings
load complete
18 / 345
loading baseball drawings
load complete
19 / 345
loading basket drawings
load complete
20 / 345
loading basketball drawings
load complete
21 / 345
loading bat drawings
load 

In [8]:
import ndjson
number = 100
labels = ["accessory", "cats", "construction", "fruit", "instrument", "one_liner", "plant", "shape", "sport", "terrain", "tool", "vehicle", "weapon", "weather", "writing_utensil"]
for label in labels:
    with open(f"Archive/{label}.ndjson") as file:
        temp = ndjson.load(file)
        i = 1
        for data in temp:
            if (i > number):
                break
            else:
                get_custom_images(data["strokes"], i, data["category"], number)
                i += 1

In [7]:
len(input_data)

34500

In [9]:
number = 100
i = 1
for drawing in input_data:
    label = drawing[0]
    data = drawing[1:][0]
    get_images(data, i, label, number)
    i += 1

In [10]:
batch_size = 32

# 70% Train
train = tf.keras.utils.image_dataset_from_directory(
    "./images",
    validation_split=0.3,
    subset = "training",
    seed = 1,
    color_mode = "grayscale",
    image_size = (32, 32),
    batch_size = batch_size,
    labels = "inferred"
)

# 30% Validate
val = tf.keras.utils.image_dataset_from_directory(
    "./images",
    validation_split = 0.3,
    subset = "validation",
    seed = 1,
    color_mode = "grayscale",
    image_size = (32, 32),
    batch_size = batch_size,
    labels = "inferred"
)

Found 36000 files belonging to 360 classes.
Using 25200 files for training.
Found 36000 files belonging to 360 classes.
Using 10800 files for validation.


In [24]:
#model architecture
model = tf.keras.Sequential()
model.add(tf.keras.layers.Rescaling(1 / 255, input_shape=(32, 32, 1)))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Conv2D(3, (3, 3), activation='relu', input_shape=(32, 32, 1)))
model.add(tf.keras.layers.MaxPooling2D((2, 2)))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Conv2D(7, (3, 3), activation='relu'))
model.add(tf.keras.layers.MaxPooling2D((2, 2)))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Conv2D(64, (3, 3), activation='relu'))
model.add(tf.keras.layers.MaxPooling2D((2, 2)))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(256, activation='relu'))
model.add(tf.keras.layers.Dense(360, activation='softmax'))

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

ValueError: Exception encountered when calling layer "conv2d_38" (type Conv2D).

Negative dimension size caused by subtracting 3 from 2 for '{{node conv2d_38/Conv2D}} = Conv2D[T=DT_FLOAT, data_format="NHWC", dilations=[1, 1, 1, 1], explicit_paddings=[], padding="VALID", strides=[1, 1, 1, 1], use_cudnn_on_gpu=true](Placeholder, conv2d_38/Conv2D/ReadVariableOp)' with input shapes: [?,2,2,32], [3,3,32,32].

Call arguments received by layer "conv2d_38" (type Conv2D):
  • inputs=tf.Tensor(shape=(None, 2, 2, 32), dtype=float32)

In [16]:
epochs = 100
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)
fit = model.fit(
    train,
    validation_data=val,
    epochs=epochs,
    verbose=1,
    callbacks = [callback]
)

Epoch 1/100
788/788 [==============================] - 9s 10ms/step - loss: 4.8846 - accuracy: 0.0739 - val_loss: 4.2790 - val_accuracy: 0.1333
Epoch 2/100
788/788 [==============================] - 8s 10ms/step - loss: 3.9354 - accuracy: 0.1712 - val_loss: 3.9153 - val_accuracy: 0.1793
Epoch 3/100
788/788 [==============================] - 8s 10ms/step - loss: 3.5936 - accuracy: 0.2197 - val_loss: 3.7412 - val_accuracy: 0.2025
Epoch 4/100
788/788 [==============================] - 8s 10ms/step - loss: 3.3718 - accuracy: 0.2514 - val_loss: 3.6278 - val_accuracy: 0.2210
Epoch 5/100
788/788 [==============================] - 8s 10ms/step - loss: 3.2022 - accuracy: 0.2754 - val_loss: 3.5812 - val_accuracy: 0.2293
Epoch 6/100
788/788 [==============================] - 8s 10ms/step - loss: 3.0643 - accuracy: 0.2991 - val_loss: 3.5573 - val_accuracy: 0.2331
Epoch 7/100
788/788 [==============================] - 8s 10ms/step - loss: 2.9462 - accuracy: 0.3159 - val_loss: 3.5430 - val_accuracy:

KeyboardInterrupt: 

In [ ]:
plt.plot(fit.history["accuracy"], label = "Train")
plt.plot(fit.history["val_accuracy"], label = "Valid")
plt.legend()
plt.title("Accuracy vs Epochs")
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.show()

In [ ]:
plt.plot(fit.history["loss"], label = "Train")
plt.plot(fit.history["val_loss"], label = "Valid")
plt.legend()
plt.title("Loss vs Epochs")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.show()

In [ ]:
model.save("google_50_image_model")
model.save("google_50_image_model.h5")